In [1]:
import sys
import os

# Thêm thư mục gốc (project/) vào sys.path
# Nếu __file__ không có (trong notebook), thay bằng:
sys.path.append(os.path.abspath(".."))

from services.products import ProductServices


In [4]:
import asyncio
import json
import re
import time
from typing import Any, Dict, List, Optional

import autogen
from autogen import ConversableAgent
from google import genai
from loguru import logger
from pydantic import BaseModel, Field

from env import env
from services.products import ProductServices
from services.search import SearchServices

In [5]:
class AgentResponse(BaseModel):
    content: str = Field(..., description="Nội dung phản hồi từ agent")
    data: Optional[Dict[str, Any]] = Field(
        default=None, description="Dữ liệu kết quả từ agent")
    status: str = Field(default="success", description="Trạng thái phản hồi")
    execution_time: Optional[float] = None

    class Config:
        from_attributes = True

client = genai.Client(api_key=env.GEMINI_API_KEY)

In [13]:
class QdrantAgent:
    def __init__(self):
        self.llm_config = self._create_llm_config()
        self.db_schema = self._get_db_schema()
        self.agent = self._create_qdrant_agent()

    def _create_llm_config(self) -> Dict[str, Any]:
        return {
            "model": "gpt-4o-mini",
            "api_key": env.OPENAI_API_KEY
        }


    def _get_db_schema(self) -> str:
        schema = """
        QDRANT COLLECTIONS (Vector Database):

        Collection: product_name_embeddings
        point = PointStruct(
            id=product_id,
            vector={"default": embedding}, # embedding của tên sản phẩm
            payload={
                "product_id": product_id,
                "name": product.name,
                "description": product.description,
            }
        )

        Collection: product_des_embeddings
        point = PointStruct(
            id=product_id,
            vector={"default": embedding}, # embedding của mô tả sản phẩm
            payload={
                "product_id": product_id,
                "name": product.name,
                "description": product.description,
            }
        )
        """
        return schema

    def _create_qdrant_agent(self) -> ConversableAgent:
        system_message = f"""
        Bạn là một chuyên gia Qdrant (vector database) với nhiệm vụ:
        1. Phân tích câu hỏi người dùng về sản phẩm, đánh giá hoặc truy vấn semantic
        2. Xác định collection Qdrant cần truy vấn và từ khóa cần nhập vào để tìm kiếm
        3. Tạo duy nhất một JSON mô tả truy vấn Qdrant

        {self.db_schema}

        Hãy trả về mô tả truy vấn Qdrant dưới dạng JSON:
        ```json
        {{
            "collection_name": "tên collection cần truy vấn",
            "payload": "giá trị đầu vào dạng chuỗi để tìm kiếm embedding",
            "limit": 5,
            "function": tên hàm sử dụng để truy vấn : search
        }}
        ```
        """
        return autogen.ConversableAgent(
            name="qdrant_expert",
            system_message=system_message,
            llm_config=self.llm_config,
            human_input_mode="NEVER"
        )

    def _extract_qdrant_query(self, response: str) -> Dict[str, Any]:
        # Cố gắng tìm đoạn JSON nằm trong khối code markdown ```json ... ```
        match = re.search(r'```json\s*(\{.*?\})\s*```', response, re.DOTALL)
        if not match:
            # Nếu không tìm thấy, thử tìm khối JSON thuần túy trong dấu ngoặc nhọn
            match = re.search(r'(\{.*?\})', response, re.DOTALL)
        if not match:
            logger.warning(f"Không tìm thấy truy vấn Qdrant: {response}")
            return {"collection_name": "products", "payload": "", "limit": 5}

        try:
            return json.loads(match.group(1))
        except json.JSONDecodeError as e:
            logger.error(f"Lỗi parse JSON: {e}")
            return {"collection_name": "products", "payload": "", "limit": 5}

    def _execute_qdrant_query(self, query_info: Dict[str, Any]) -> List[Dict]:
        function = query_info.get("function")

        if function == "search":
            query_text = query_info.get("payload", "")
            return SearchServices.search(payload=query_text, collection_name=query_info.get("collection_name"), limit=query_info.get("limit", 5))

        elif function == "recommend_for_user":
            user_id = query_info.get("user_id")
            return SearchServices.search(payload=query_info.get("payload", ""), collection_name="user_queries", limit=query_info.get("limit", 5))

        else:
            return SearchServices.search(payload=query_info.get("payload", ""), collection_name=query_info.get("collection_name"), limit=query_info.get("limit", 5))

    def _generate_explanation(self, query_info: Dict[str, Any], query_result: List[Dict], user_query: str) -> str:
        if not query_result:
            return "Không tìm thấy kết quả phù hợp với yêu cầu của bạn."

        data_description = f"Tìm thấy {len(query_result)} kết quả phù hợp."
        top_products = ", ".join(
            f"{p['name']} ({p['price']} VND)" for p in query_result[:3]
        )
        data_description += f" Ví dụ: {top_products}."
        print(f"✅✅💣💣➡️❗Data description: {data_description}")
        explanation_prompt = f"""
        Câu hỏi của người dùng: {user_query}

        Mô tả dữ liệu trả về: {data_description}

        Hãy viết câu trả lời thân thiện bằng tiếng Việt, không đề cập đến truy vấn hay kỹ thuật vector.
        """


        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents= explanation_prompt,
        )
        print(f"❎✅❤️😍👍✌️🤷‍♂️🤷‍♀️🤦‍♂️💕response.text: {response.text}")
        return response.text


    async def process_query(self, user_query: str, user_id: Optional[int] = None) -> AgentResponse:
        # start_time = time.time()
        # try:
            prompt = f"""
            Hãy phân tích và tạo truy vấn Qdrant cho câu hỏi sau:
            "{user_query}"
            {f"Người dùng: {user_id}" if user_id else ""}
            """

            agent_response = await self.agent.a_generate_reply(messages=[{"role": "user", "content": prompt}])
            # print(f"Agent response: {agent_response}")
            query_info = self._extract_qdrant_query(agent_response)
            # print(f"✅✅💣💣➡️❗Query info: {query_info}")
            
            query_info["user_id"] = user_id

            result = self._execute_qdrant_query(query_info)
            
            products = []
            for id in result:
                product = ProductServices.get(id)
                if product:
                    products.append(product)
            result = products
            print(f"✅✅💣💣➡️❗Query result: {result[0]}")
            return result
            explanation = self._generate_explanation(query_info, result, user_query)
            return AgentResponse(
                content=explanation,
                data={"query_info": query_info, "results": result, "count": len(result)},
                status="success",
                execution_time=time.time() - start_time
            )

        # except Exception as e:
        #     logger.error(f"Lỗi truy vấn Qdrant: {e}")
        #     return AgentResponse(
        #         content="Đã xảy ra lỗi khi thực hiện truy vấn.",
        #         data=None,
        #         status="error",
        #         execution_time=time.time() - start_time
        #     )

In [17]:
agent = QdrantAgent()
response = await agent.process_query("sách nói về cách sống trong rừng trong rú", user_id=1)

✅✅💣💣➡️❗Query result: product_id=276182805 name='Sách nói Fonos: Trí Tuệ Của Rừng' product_short_url='https://tiki.vn/product-p276182805.html?spid=276182806' description='<p>Hạn sử dụng: 2025-09-20 00:00:00</p><p><strong>Giới thiệu nội dung</strong><br />Bắt đầu từ nỗi lo lắng sâu thẳm về tương lai của rừng trên Trái Đất, Suzanne Simard – lúc bấy giờ chỉ là nhân viên của một công ty khai thác gỗ – đã dấn bước vào một hành trình nghiên cứu, tìm tòi phương pháp để cứu lấy những cánh rừng. Hành trình ấy hóa ra lại là khởi đầu cho một công trình cả đời của bà, khai mở những bí mật của rừng, đặc biệt là hệ thống mạng lưới thông tin phức tạp dưới lòng đất giữa các cây. Chuyến phiêu lưu ấy được liên tục tiếp năng lượng từ tính cách tò mò đã có từ tuổi thơ của Simard.</p>\n<p>Trí Tuệ Của Rừng như một cuốn phim ghi lại những động lực, trăn trở, thành công, gục ngã, tình yêu và nước mắt của Giáo sư – Nhà khoa học Suzanne Simard. Những trang viết gần gũi và đong đầy tình cảm đã trở thành lời vẫy g

In [ ]:
response[0].__dict__['name']

'Sách nói Fonos: Trí Tuệ Của Rừng'